In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import seaborn as sns
con = sqlite3.connect('database.sqlite')
filtered_data=pd.read_sql_query(
"""
    SELECT*
    FROM Reviews
    WHERE Score!=3
    """,con)
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

actualscore=filtered_data['Score']
positivenegative=actualscore.map(partition)
filtered_data['Score']=positivenegative
print(filtered_data)

            Id   ProductId          UserId                      ProfileName  \
0            1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1            2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2            3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3            4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4            5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...        ...         ...             ...                              ...   
525809  568450  B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
525810  568451  B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
525811  568452  B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
525812  568453  B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
525813  568454  B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

        HelpfulnessNumerator  HelpfulnessDenominato

In [6]:
sorted_data=filtered_data.sort_values('Time',axis=0,ascending=True)
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final=final.head(2000)
print(final)

            Id   ProductId          UserId                 ProfileName  \
138706  150524  0006641040   ACITT7DI6IDDL             shari zychinski   
138683  150501  0006641040   AJ46FKXOVC7NR          Nicholas A Mesiano   
417839  451856  B00004CXX9   AIUWLEQ1ADEG5            Elizabeth Medina   
417859  451878  B00004CXX9  A344SMIA5JECGM             Vincent P. Ross   
417838  451855  B00004CXX9   AJH6LUC1UT1ON    The Phantom of the Opera   
...        ...         ...             ...                         ...   
339378  367180  B0006U910O  A2RAZWCD8AKIY0                   M. Buford   
433899  469222  B000AQFQC6  A33CIMYZ9032X7                  D. Tierney   
32837    35743  B00020UGN4  A1GIJSR25ENRUR        Jonathan C. Fogelman   
524611  567159  B0000AHA22  A2VQ6C6C4Q0K4G             N.K. "buenamom"   
425776  460476  B0001CVDZI  A3JGPJ7UTGM1WJ  Mouth on Fire "jerk lover"   

        HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
138706                     0     

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
tf_idf_vect=TfidfVectorizer(ngram_range=(2,2),analyzer='word',stop_words='english')
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)
final_tf_idf=final_tf_idf.toarray()
print(final_tf_idf)
print(final_tf_idf.shape)

NameError: name 'final' is not defined

In [8]:
X=final_tf_idf[:,:]
y=final['Score']
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train,X_test = np.split(X, [int(0.8*len(X))])
y_train,y_test = np.split(y, [int(0.8*len(y))])
X_train,X_cv = np.split(X_train, [int(0.75*len(X_train))])
y_train,y_cv = np.split(y_train, [int(0.75*len(y_train))])
from sklearn.neighbors import KNeighborsClassifier
neighbors=np.arange(1,50,2)
cv_scores=[]
for i in neighbors:
    knn=KNeighborsClassifier(n_neighbors=i)
    scores=cross_val_score(knn,X_cv,y_cv,cv=10,scoring='accuracy')
    cv_scores.append(scores.mean())
    
MSE=[1-x for x in cv_scores]
optimal_k=neighbors[MSE.index(min(MSE))]
print(optimal_k)
    
    
    


5


In [10]:
optimal_knn=KNeighborsClassifier(n_neighbors=optimal_k)
optimal_knn.fit(X_train,y_train)
pred=optimal_knn.predict(X_test)
acc=accuracy_score(y_test,pred)*100
print(acc)

79.0
